In [26]:
import pandas as pd
import numpy as np

from datetime import datetime as dt
from functools import partial, wraps

from plasticc.featurize import process_meta, featurize
from plasticc.training import path_from_cv_score, train_and_validate
from plasticc.final import featurize_test, predict_test

In [5]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [38]:
%run ../plasticc/training.py

In [17]:
fcp = {
    'flux': {
        'longest_strike_above_mean': None,
        'longest_strike_below_mean': None,
        'mean_change': None,
        'mean_abs_change': None,
        'length': None,
    },

    'flux_by_flux_ratio_sq': {
        'longest_strike_above_mean': None,
        'longest_strike_below_mean': None,       
    },

    'flux_passband': {
        'fft_coefficient': [
                {'coeff': 0, 'attr': 'abs'}, 
                {'coeff': 1, 'attr': 'abs'}
            ],
        'kurtosis' : None, 
        'skewness' : None,
    },

    'mjd': {
        'maximum': None, 
        'minimum': None,
        'mean_change': None,
        'mean_abs_change': None,
    },
}

In [18]:
aggs = {
    'flux': ['min', 'max', 'mean', 'median', 'std', 'skew'],
    'flux_err': ['min', 'max', 'mean', 'median', 'std', 'skew'],
    'detected': ['mean'],
    'flux_ratio_sq':['sum', 'skew'],
    'flux_by_flux_ratio_sq':['sum','skew'],
}

### Generate features for model training

In [20]:
%%time
meta_train = process_meta('../data/raw/training_set_metadata.csv')
train = pd.read_csv('../data/raw/training_set.csv')

CPU times: user 1.59 s, sys: 2.63 s, total: 4.22 s
Wall time: 4.32 s


In [21]:
%%time
X = featurize(train, meta_train, aggs, fcp, n_jobs=4)

Feature Extraction: 100%|██████████| 20/20 [00:01<00:00, 12.43it/s]


CPU times: user 24.7 s, sys: 23.4 s, total: 48.1 s
Wall time: 53.1 s


In [22]:
if 'target' in X:
    y = X['target']
    del X['target']
else:
    print("What the duck")
    3//0

In [23]:
%%time
train_mean = X.mean(axis=0)
#train_mean.to_hdf('train_data.hdf5', 'data')
# pd.set_option('display.max_rows', 500)
#import pdb; pdb.set_trace()
X.fillna(0, inplace=True)

CPU times: user 12 ms, sys: 0 ns, total: 12 ms
Wall time: 10.8 ms


### Train model with CV

In [39]:
lgbm_params = {
    'device': 'cpu',
    'objective': 'multiclass',
    'num_class': 14,
    'boosting_type': 'gbdt',
    'n_jobs': 16,
    'max_depth': 6,
    'n_estimators': 1024,
    'subsample_freq': 2,
    'subsample_for_bin': 5000,
    'min_data_per_group': 100,
    'max_cat_to_onehot': 4,
    'cat_l2': 1.0,
    'cat_smooth': 59.5,
    'max_cat_threshold': 32,
    'metric_freq': 10,
    'verbosity': -1,
    'metric': 'multi_logloss',
    'xgboost_dart_mode': False,
    'uniform_drop': False,
    'colsample_bytree': 0.5,
    'drop_rate': 0.173,
    'learning_rate': 0.0267,
    'max_drop': 5,
    'min_child_samples': 10,
    'min_child_weight': 100.0,
    'min_split_gain': 0.126,
    'num_leaves': 7,
    'reg_alpha': 0.1,
    'reg_lambda': 0.00023,
    'skip_drop': 0.44,
    'subsample': 0.75,
    'max_bin': 32,
    'min_data_in_leaf': 13,
    'lambda_l1': 2,
}


xgb_params = {
    'objective': 'multiclass',
    'booster': 'gbdtree',
    'n_jobs': 16,
    'max_depth': 7,
    'n_estimators': 1024,
    'verbosity': -1,
    'colsample_bytree': 0.5,
    'learning_rate': 0.0267,
    'min_child_weight': 100.0,
    'reg_alpha': 0.1,
    'reg_lambda': 0.00023,
    'subsample': 0.75
}


In [40]:
colnames_to_ignore = set([
    'object_id',
    'hostgal_specz',
    'ra',
    'decl',
    'gal_l',
    'gal_b',
    'ddf'
])
feature_colnames = [col for col in X.columns if col not in colnames_to_ignore]
id_colname = 'object_id'

In [43]:
%%time
# modeling from CV
clfs, score, importances = train_and_validate(
    X=X, 
    y=y, 
    feature_colnames=feature_colnames, 
    id_colname=id_colname, 
    model='lgbm', 
    model_params={
        'lgbm': lgbm_params
    },
    nr_fold=5, 
    random_state=1
)

Training until validation scores don't improve for 50 rounds.
[100]	training's multi_logloss: 0.484835	training's wloss: 0.471557	valid_1's multi_logloss: 0.872412	valid_1's wloss: 0.710658
[200]	training's multi_logloss: 0.343904	training's wloss: 0.330327	valid_1's multi_logloss: 0.789207	valid_1's wloss: 0.701091
Early stopping, best iteration is:
[174]	training's multi_logloss: 0.370746	training's wloss: 0.356952	valid_1's multi_logloss: 0.803366	valid_1's wloss: 0.690914
no 1-fold loss: 0.6909142213288597
Training until validation scores don't improve for 50 rounds.
[100]	training's multi_logloss: 0.479734	training's wloss: 0.468144	valid_1's multi_logloss: 0.864813	valid_1's wloss: 0.745861
Early stopping, best iteration is:
[107]	training's multi_logloss: 0.465315	training's wloss: 0.453538	valid_1's multi_logloss: 0.855136	valid_1's wloss: 0.742924
no 2-fold loss: 0.7429237262592194
Training until validation scores don't improve for 50 rounds.
[100]	training's multi_logloss: 0.

In [12]:
importances.sort_values(by='mean_gain', ascending=False).head(15)

,feature,gain,fold,mean_gain
55,hostgal_photoz,1838,3,1886.4
55,hostgal_photoz,1788,4,1886.4
55,hostgal_photoz,1821,2,1886.4
55,hostgal_photoz,1949,1,1886.4
55,hostgal_photoz,2036,5,1886.4
54,mjd_diff_det,1729,5,1636.2
54,mjd_diff_det,1649,4,1636.2
54,mjd_diff_det,1658,1,1636.2
54,mjd_diff_det,1529,2,1636.2
54,mjd_diff_det,1616,3,1636.2


In [13]:
submission_file_path = path_from_cv_score(score)
submission_file_path

'/home/kk385830/astronomical-classification/submissions/subm_0.655012_2018-12-10-15-50.csv'

### Test set features

In [14]:
chunk_size_one_100 = 453653105 // 100 + 1
# chunk_size_one_100 = 1000001 // 100 + 1
chunk_size_one_100

4536532

In [ ]:
featurized_set_savior = 2//3  # to prevent accidental running
%%time
# should take 100x (time after 1st iteration)
featurize_test(
    featurize_configs={'aggs': aggs, 'fcp': fcp}, 
    n_jobs=12,
    meta_path='../data/raw/test_set_metadata.csv',
    test_path='../data/raw/test_set.csv',
    output_path='../data/features/test-all-feat-from-kernel-repro.csv',
    id_colname='object_id',
    chunks=chunk_size_one_100,  # alternatively: 5000000
)

Feature Extraction: 100%|██████████| 60/60 [00:01<00:00, 37.97it/s]


        4536532 done in   1.0 minutes


Feature Extraction: 100%|██████████| 60/60 [00:01<00:00, 38.33it/s]


        9073064 done in   1.9 minutes


Feature Extraction: 100%|██████████| 60/60 [00:03<00:00, 18.97it/s]


...

      367459092 done in 146.8 minutes


Feature Extraction: 100%|██████████| 60/60 [00:03<00:00, 15.05it/s]


      371995624 done in 148.6 minutes


Feature Extraction: 100%|██████████| 60/60 [00:03<00:00, 17.05it/s]


      376532156 done in 150.3 minutes


### Test set predictions

In [20]:
%%time
submission = predict_test(
    clfs=clfs, 
    feature_colnames=feature_colnames, 
    id_colname=id_colname, 
    input_path='../data/features/test-all-feat-from-kernel-repro.csv', 
    output_path=submission_file_path, 
    verbose=True
)

Loading data...


  0%|          | 0/5 [00:00<?, ?it/s]

Generating predictions...


100%|██████████| 5/5 [06:59<00:00, 85.82s/it]


Postprocessing...
Submission shape before grouping: (3492890, 16)
Submission shape after grouping: (3492890, 15)
Submission shape after postprocessing: (3492890, 15)
Validating submission file...
Saving submission...
Submission saved to f/home/kk385830/astronomical-classification/submissions/subm_0.655012_2018-12-10-15-50.csv
CPU times: user 1h 45min 8s, sys: 1min 31s, total: 1h 46min 40s
Wall time: 9min 13s


In [21]:
submission.head()

,class_6,class_15,class_16,class_42,class_52,class_53,class_62,class_64,class_65,class_67,class_88,class_90,class_92,class_95,class_99
object_id,,,,,,,,,,,,,,,
13,0.000360,0.002533,0.000253,0.398584,0.234868,0.000588,0.076150,0.000179,0.000245,0.002449,0.000558,0.142123,0.000271,0.000576,0.140263
14,0.000729,0.017038,0.000646,0.131502,0.041601,0.001021,0.071493,0.001367,0.001644,0.021206,0.004001,0.582138,0.000522,0.008279,0.116813
17,0.000281,0.003488,0.000267,0.029676,0.018126,0.000389,0.017293,0.001509,0.000419,0.039358,0.004025,0.822417,0.000155,0.004335,0.058262
23,0.000532,0.002382,0.000454,0.050927,0.015834,0.000628,0.060551,0.003201,0.000719,0.240286,0.001429,0.465966,0.000241,0.023801,0.133049
34,0.000352,0.004464,0.000237,0.099490,0.163679,0.000487,0.037197,0.000272,0.000470,0.012618,0.000360,0.559497,0.000196,0.000489,0.120193
